## T2 Calculations

In [ ]:
data_dict = import_csv_files(r".", "Cu")
cu_data = []

for m_value, df in data_dict.items():
    t_2, r_squared = find_t2(df)
    cu_data.append((m_value, t_2, r_squared))
    print(f'Cu Concentration: {m_value:.4e} \n\tT_2: {t_2:.4e} \n\t R^2: {r_squared:.3f}')

data_dict = import_csv_files(r".", "Fe")
fe_data = []

for m_value, df in data_dict.items():
    t_2, r_squared = find_t2(df)
    fe_data.append((m_value, t_2, r_squared))
    print(f'Fe Concentration: {m_value:.4e} \n\tT_2: {t_2:.4e} \n\t R^2: {r_squared:.3f}')

C:\Users\volca\AppData\Local\Temp\ipykernel_18084\923934287.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  time_data[i, 1] = delta_t


Cu Concentration: 4.8000e-02 
	T_2: 5.0603e-01 
	 R^2: 0.953
Cu Concentration: 8.0000e-02 
	T_2: 7.4547e-01 
	 R^2: 0.974
Cu Concentration: 1.0000e-01 
	T_2: 8.5592e-01 
	 R^2: 0.975
Cu Concentration: 1.6000e-01 
	T_2: 1.3914e+00 
	 R^2: 0.991
Cu Concentration: 2.5000e-01 
	T_2: 2.2840e+00 
	 R^2: 0.991
Cu Concentration: 3.2000e-01 
	T_2: 3.2720e+00 
	 R^2: 0.990
Cu Concentration: 4.8000e-01 
	T_2: 4.5762e+00 
	 R^2: 0.994
Cu Concentration: 6.6000e-01 
	T_2: 7.0840e+00 
	 R^2: 0.986
Cu Concentration: 7.5000e-01 
	T_2: 8.5366e+00 
	 R^2: 0.959


C:\Users\volca\AppData\Local\Temp\ipykernel_18084\923934287.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  time_data[i, 1] = delta_t


Fe Concentration: 2.5000e-03 
	T_2: -7.0178e-02 
	 R^2: 0.004
Fe Concentration: 5.0000e-03 
	T_2: 3.1329e-01 
	 R^2: 0.823
Fe Concentration: 1.0000e-02 
	T_2: 4.8125e-01 
	 R^2: 0.944
Fe Concentration: 1.5000e-02 
	T_2: 9.9888e-01 
	 R^2: 0.990
Fe Concentration: 2.5000e-02 
	T_2: 1.9692e+00 
	 R^2: 0.992
Fe Concentration: 3.0000e-02 
	T_2: 2.4700e+00 
	 R^2: 0.994
Fe Concentration: 4.0000e-02 
	T_2: 3.0227e+00 
	 R^2: 0.994
Fe Concentration: 5.0000e-02 
	T_2: 4.9577e+00 
	 R^2: 0.910


## T1 Calulations

## T1 Validation

In [ ]:
def compute_chi_squared(arr, popt, sigma_y):
    local_arr = np.permute_dims(arr)
    t = local_arr[:, 0]
    y = local_arr[:, 1]
    A, B, T1, C = popt
    y_fit = ir_model(t, A, B, T1, C)

    residuals = y - y_fit
    chi2 = np.sum((residuals / sigma_y) ** 2)
    dof = len(t) - len(popt)
    chi2_red = chi2 / dof
    return chi2, chi2_red

def fit_and_plot(dict_of_dicts, out_filename=None, figsize=(12,9),
                 cmap_name='viridis'):
    """
    dict_of_dicts: mapping label -> data_dict (concentration->array (N,2))
    q_mv: ADC full-scale in mV (default 20 mV -> q = 20e-3 / 256)
    quant_steps: number of quant steps representing the per-point uncertainty (user stated 5)
    Returns: t1_results = {label: {conc: (T1_sec, T1_err_sec), ...}, ...}
    """
    cmap = cm.get_cmap(cmap_name)

    species = list(dict_of_dicts.keys())

    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 0.7], hspace=0.35, wspace=0.28)
    ax_top_left = fig.add_subplot(gs[0,0])
    ax_top_right = fig.add_subplot(gs[0,1])
    ax_bot_left = fig.add_subplot(gs[1,0])
    ax_bot_right = fig.add_subplot(gs[1,1])
    axs_top = [ax_top_left, ax_top_right]
    axs_bot = [ax_bot_left, ax_bot_right]

    t1_results = {}

    time_scale = 1000.0   # display t in ms
    amp_scale = 1000.0    # display amp in mV
    fig.suptitle('$T_1$ Data and Regressions')
    for i, label in enumerate(species):
        ax = axs_top[i] if i < 2 else axs_top[-1]
        ax.set_title(f'{label} Echo Time to Delay Time')
        ax.set_xlabel("Delay time (ms)")
        ax.set_ylabel("Echo amplitude (mV)")
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth=0.6, alpha=0.6)
        ax.grid(which='minor', linestyle=':', linewidth=0.5, alpha=0.4)

        data_dict = dict_of_dicts[label]
        t1_results[label] = {}

        if not data_dict:
            continue

        concs_sorted = np.array(sorted(map(float, data_dict.keys())))
        norm = Normalize(vmin=concs_sorted.min(), vmax=concs_sorted.max(), clip=True)
        print(label)
        for conc in sorted(map(float, data_dict.keys())):
            arr = np.asarray(data_dict[conc])
            
            popt, perr, chi2_red, dof, t, y = fit_dataset(arr, sigma_y = amp_stdev)

            A, B, T1, C = popt
            A_err, B_err, T1_err, C_err = perr if perr is not None else (np.nan, np.nan, np.nan, np.nan)

            t_smooth = np.linspace(t.min(), t.max(), 300)
            y_fit = ir_model(t_smooth, *popt)
            color = cmap(norm(conc))

            ax.scatter(time_scale * t, amp_scale * y, s=30, marker='o', color=color,
                       alpha=0.9, edgecolor='k', linewidth=0.3, zorder=2)
            ax.plot(time_scale * t_smooth, amp_scale * y_fit, color=color, linewidth=1.6, zorder=1)

            t1_results[label][conc] = (float(T1), float(T1_err))

            chi2, chi2_red = compute_chi_squared(arr, popt, sigma_y=amp_stdev)
            print(f'T1 for concentration: {conc:.3e} M')
            print(f"\tChi² = {chi2:.3e}, Reduced Chi² = {chi2_red:.3f}")


    lin_fit_stats = {}
    # Bottom row: T1 vs concentration with vertical error bars and weighted linear fit
    for i, label in enumerate(species):
        axb = axs_bot[i]
        axb.set_xlabel("Concentration (M)")
        axb.set_ylabel("T1 (ms)")
        axb.minorticks_on()
        axb.grid(which='major', linestyle='-', linewidth=0.6, alpha=0.6)
        axb.grid(which='minor', linestyle=':', linewidth=0.5, alpha=0.4)

        results = t1_results.get(label, {})
        if not results:
            continue

        concs = np.array(sorted(results.keys()), dtype=float)
        T1s_s = np.array([results[c][0] for c in concs], dtype=float)
        T1_errs_s = np.array([results[c][1] for c in concs], dtype=float)

        # convert to ms for plotting and fitting
        T1s_ms = T1s_s * 1000.0
        T1_errs_ms = T1_errs_s * 1000.0

        norm = Normalize(vmin=concs.min(), vmax=concs.max(), clip=True)


        axb.scatter(concs, T1s_ms, c=concs, cmap=cmap, norm=norm,
                        s=60, edgecolor='k', linewidth=0.3, zorder=2)

        good_mask = np.isfinite(T1s_ms) & np.isfinite(concs)
        chi2_lin = np.nan
        chi2_red_lin = np.nan
        m = b = m_err = b_err = r2 = np.nan

        
        x = concs[good_mask]
        y = T1s_ms[good_mask]

        idx = np.where(good_mask)[0]
        xw = concs[idx] 
        yw = T1s_ms[idx]
        yw_err = T1_errs_ms[idx]

        yw_err_safe = np.maximum(yw_err, 1e-12)

        popt_pow, pcov_pow = curve_fit(
            lambda X, A, B: A * np.power(X, B),
            xw, yw,
            sigma=yw_err_safe,
            absolute_sigma=True
        )
        a, b = popt_pow
        a_err, b_err = np.sqrt(np.diag(pcov_pow))

        # chi2 on points used in fit
        yhat = a * np.power(xw, b)
        chi2_lin = np.sum(((yw - yhat) / yw_err_safe) ** 2)
        p = 2
        dof_lin = max(0, xw.size - p)
        chi2_red_lin = chi2_lin / dof_lin if dof_lin > 0 else np.nan

        # R^2 computed on the same good points
        ss_res = np.nansum((yw - yhat) ** 2)
        ss_tot = np.nansum((yw - np.nanmean(yw)) ** 2)
        r2 = 1.0 - ss_res / ss_tot if ss_tot != 0 else np.nan

        # plot fitted curve (log-log)
        x_min = np.min(concs[concs > 0])
        x_max = np.max(concs)
        x_line = np.logspace(np.log10(x_min), np.log10(x_max), 200)
        axb.loglog(x_line, a * np.power(x_line, b), '--', color='gray', linewidth=1.6)

        lin_fit_stats[label] = {'Chi2' : chi2_lin, 'Chi2_red' : chi2_red_lin, 
                                'a' : a, 'b' : b, 
                                'delta a' : a_err, 'delta b': b_err, 'r2' : r2}

        cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
                            ax=axb, orientation='vertical', pad=0.06)
        cbar.set_label("Concentration (M)")
        axb.set_title(f'Weighted Reg. $T_1$ to Con. $\hat{{y}} = {a:.4f}x^{{{b:.3}}}$')
    if out_filename:
        fig.savefig(out_filename, bbox_inches='tight', dpi=200)

    return t1_results, lin_fit_stats


<>:164: SyntaxWarning: "\h" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\h"? A raw string is also an option.
<>:164: SyntaxWarning: "\h" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\h"? A raw string is also an option.
C:\Users\volca\AppData\Local\Temp\ipykernel_20096\1735587385.py:164: SyntaxWarning: "\h" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\h"? A raw string is also an option.
  axb.set_title(f'Weighted Reg. $T_1$ to Con. $\hat{{y}} = {a:.4f}x^{{{b:.3}}}$')


In [ ]:
results_dict, conc_reg_stats = fit_and_plot({'Cu': cu_amp_data, 'Fe': fe_amp_data})

In [48]:
from pprint import pprint
pprint(conc_reg_stats)

{'Cu': {'Chi2': np.float64(0.0005949814785659811),
        'Chi2_red': np.float64(8.499735408085444e-05),
        'a': np.float64(0.41708466325202853),
        'b': np.float64(-1.6426220334785622),
        'delta a': np.float64(9.036580868692491),
        'delta b': np.float64(61.6438892810174),
        'r2': np.float64(-1.762684944593817)},
 'Fe': {'Chi2': np.float64(0.0020675291559189212),
        'Chi2_red': np.float64(0.0003445881926531535),
        'a': np.float64(0.07756571078971329),
        'b': np.float64(-0.8660057487683235),
        'delta a': np.float64(1.8249365409295695),
        'delta b': np.float64(7.269238947171127),
        'r2': np.float64(-0.25439117633275554)}}
